In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D, LSTM, RepeatVector


# Load data
data = pd.read_excel('D:\\finalyear\\sine-wave-denoise\\Dataset\\ex1.xlsx')
original_amplitude = data['Amplitude'].values
noised_amplitude = data['Amplitude_noise'].values

# Reshape and scale data
original_amplitude = original_amplitude.reshape(-1, 1)
noised_amplitude = noised_amplitude.reshape(-1, 1)

scaler = MinMaxScaler()
original_amplitude = scaler.fit_transform(original_amplitude)
noised_amplitude = scaler.transform(noised_amplitude)

# Windowing function
window_size = 4500

def create_windows(data, window_size):
    windows = []
    for i in range(len(data) - window_size + 1):
        windows.append(data[i:i + window_size])
    return np.array(windows)

X = create_windows(noised_amplitude, window_size)
y = create_windows(original_amplitude, window_size)

# Reshape for training
X_train = X.reshape(-1, window_size, 1)
y_train = y.reshape(-1, window_size, 1)


In [5]:

# Conv1D Autoencoder Model
input_signal = Input(shape=(window_size, 1))
x = Conv1D(16, 3, activation='relu', padding='same')(input_signal)
x = MaxPooling1D(2, padding='same')(x)
x = Conv1D(8, 3, activation='relu', padding='same')(x)
encoded = MaxPooling1D(2, padding='same')(x)

x = Conv1D(8, 3, activation='relu', padding='same')(encoded)
x = UpSampling1D(2)(x)
x = Conv1D(16, 3, activation='relu', padding='same')(x)
x = UpSampling1D(2)(x)
decoded = Conv1D(1, 3, activation='relu', padding='same')(x)

autoencoder = Model(input_signal, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train Conv1D Autoencoder
autoencoder.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)
denoised_signal = autoencoder.predict(X_train)
denoised_signal = scaler.inverse_transform(denoised_signal.reshape(-1, 1))


Epoch 1/20
7/7 [==============================] - 198s 1s/step - loss: 0.3618 - val_loss: 0.3390
Epoch 2/20
7/7 [==============================] - 3s 393ms/step - loss: 0.2834 - val_loss: 0.2061
Epoch 3/20
7/7 [==============================] - 3s 406ms/step - loss: 0.1532 - val_loss: 0.0860
Epoch 4/20
7/7 [==============================] - 3s 393ms/step - loss: 0.0557 - val_loss: 0.0324
Epoch 5/20
7/7 [==============================] - 3s 382ms/step - loss: 0.0373 - val_loss: 0.0388
Epoch 6/20
7/7 [==============================] - 3s 399ms/step - loss: 0.0325 - val_loss: 0.0264
Epoch 7/20
7/7 [==============================] - 3s 394ms/step - loss: 0.0270 - val_loss: 0.0254
Epoch 8/20
7/7 [==============================] - 3s 389ms/step - loss: 0.0224 - val_loss: 0.0192
Epoch 9/20
7/7 [==============================] - 3s 390ms/step - loss: 0.0181 - val_loss: 0.0151
Epoch 10/20
7/7 [==============================] - 3s 404ms/step - loss: 0.0138 - val_loss: 0.0117
Epoch 11/20
7/7 [===

In [6]:

# LSTM Autoencoder Model
input_signal_lstm = Input(shape=(window_size, 1))
encoded_lstm = LSTM(64, activation='relu', return_sequences=True)(input_signal_lstm)
encoded_lstm = LSTM(32, activation='relu', return_sequences=False)(encoded_lstm)

decoded_lstm = RepeatVector(window_size)(encoded_lstm)
decoded_lstm = LSTM(32, activation='relu', return_sequences=True)(decoded_lstm)
decoded_lstm = LSTM(64, activation='relu', return_sequences=True)(decoded_lstm)
decoded_lstm = Conv1D(1, 3, activation='relu', padding='same')(decoded_lstm)

lstm_autoencoder = Model(input_signal_lstm, decoded_lstm)
lstm_autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train LSTM Autoencoder
lstm_autoencoder.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2)
denoised_signal_lstm = lstm_autoencoder.predict(X_train)
denoised_signal_lstm = scaler.inverse_transform(denoised_signal_lstm.reshape(-1, 1))


Epoch 1/20
7/7 [==============================] - 320s 43s/step - loss: 0.3561 - val_loss: 0.3342
Epoch 2/20
7/7 [==============================] - 258s 37s/step - loss: 0.2945 - val_loss: 0.2085
Epoch 3/20
7/7 [==============================] - 261s 38s/step - loss: 0.1790 - val_loss: 0.1311
Epoch 4/20
7/7 [==============================] - 244s 34s/step - loss: 0.1487 - val_loss: 0.1359
Epoch 5/20
7/7 [==============================] - 215s 31s/step - loss: 0.1387 - val_loss: 0.1313
Epoch 6/20
2/7 [=======>......................] - ETA: 3:16 - loss: 0.1369

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint


# Define the input layer
input_signal = Input(shape=(4500, 1))

# Build the convolutional layers
x = Conv1D(16, kernel_size=3, activation='relu', padding='same')(input_signal)
x = MaxPooling1D(pool_size=2, padding='same')(x)
x = Conv1D(8, kernel_size=3, activation='relu', padding='same')(x)
encoded = MaxPooling1D(pool_size=2, padding='same')(x)

# Build the upsampling layers
x = Conv1D(8, kernel_size=3, activation='relu', padding='same')(encoded)
x = UpSampling1D(size=2)(x)
x = Conv1D(16, kernel_size=3, activation='relu', padding='same')(x)
x = UpSampling1D(size=2)(x)
decoded = Conv1D(1, kernel_size=3, activation='relu', padding='same')(x)

# Compile the model
conv2_autoencoder = Model(inputs=input_signal, outputs=decoded)
conv2_autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')


# Define checkpoint callback
checkpoint_path = "checkpoints/denoiser_model.h5"
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)





Epoch 1/20
7/7 [==============================] - 107s 668ms/step - loss: 0.3538 - val_loss: 0.2962
Epoch 2/20
7/7 [==============================] - 3s 370ms/step - loss: 0.2352 - val_loss: 0.1546
Epoch 3/20
7/7 [==============================] - 3s 455ms/step - loss: 0.1085 - val_loss: 0.0534
Epoch 4/20
7/7 [==============================] - 4s 590ms/step - loss: 0.0381 - val_loss: 0.0341
Epoch 5/20
7/7 [==============================] - 3s 464ms/step - loss: 0.0373 - val_loss: 0.0311
Epoch 6/20
7/7 [==============================] - 4s 518ms/step - loss: 0.0253 - val_loss: 0.0204
Epoch 7/20
7/7 [==============================] - 3s 441ms/step - loss: 0.0201 - val_loss: 0.0166
Epoch 8/20
7/7 [==============================] - 3s 502ms/step - loss: 0.0135 - val_loss: 0.0087
Epoch 9/20
7/7 [==============================] - 4s 560ms/step - loss: 0.0066 - val_loss: 0.0031
Epoch 10/20
7/7 [==============================] - 4s 522ms/step - loss: 0.0027 - val_loss: 0.0016
Epoch 11/20
7/7 [

In [ ]:
# Train the model

conv2_autoencoder.fit(X_train, y_train,
                      epochs=20,
                      batch_size=64,
                      validation_split=0.2,
                      callbacks=[checkpoint])

                      
# Predict using the trained (or best saved) model
denoised_signal_conv2 = conv2_autoencoder.predict(X_train)

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

# ✅ Load your trained model
model = load_model("checkpoints/denoiser_model.h5")

# ✅ Prepare your input (reshape as needed)
# Example: assume 'data["Amplitude_noise"]' is the noisy signal
X_noisy = data['Amplitude_noise'].values.reshape(1, -1, 1)  # shape: (1, time_steps, 1)

# ✅ Predict denoised output
denoised_signal_conv2 = model.predict(X_noisy)

# ✅ Flatten for plotting
denoised_signal_conv2 = denoised_signal_conv2.flatten()


In [3]:

# Visualization
time_steps = np.arange(len(data))
plt.figure(figsize=(15, 10))
plt.plot(time_steps, data['Amplitude'], label='Original Amplitude')
plt.plot(time_steps, data['Amplitude_noise'], label='Noised Amplitude')
plt.plot(time_steps, denoised_signal.flatten()[:len(data)], label='Conv1D Autoencoder Denoised')
plt.plot(time_steps, denoised_signal_lstm.flatten()[:len(data)], label='LSTM Autoencoder Denoised')
plt.plot(time_steps, denoised_signal_conv2.flatten()[:len(data)], label='Enhanced Conv1D Autoencoder Denoised')

plt.xlabel('Time Steps')
plt.ylabel('Amplitude')
plt.title('Signal Denoising Comparison')
plt.legend()
plt.show()


NameError: name 'np' is not defined